In [1]:
from xml.dom import minidom

# parse an xml file by name
mydoc = minidom.parse('/home/neil/Neil_Work/MS_SJSU/295A/dataset/useful_dataset/data/Posts.xml')

In [2]:
rows = mydoc.getElementsByTagName('row')

In [3]:
qa = []
i = 0
for row in rows:
    if row.hasAttribute('AcceptedAnswerId') is True or row.hasAttribute('ParentId') is True:
        qa.insert(i,row)
        i = i + 1

print(i)

73389


In [4]:
qa2 = {}
k=0
for a in qa:
    temp = {} # 0: id, 1: acceptedanswerid, 2:parentid, 3: score, 4: body, 5: title, 6: tags
    j = 0
    names = ['Id','AcceptedAnswerId','ParentId','Score','Body','Title','Tags']
    for eachName in names:
        temp[eachName]=a.attributes[names[j]].value if a.hasAttribute(names[j]) is True else ''
        j = j + 1
    
    qa2[temp['Id']]=temp
    k = k + 1

In [5]:
qa2['1']

{'AcceptedAnswerId': '13',
 'Body': '<p>This is a common question by those who have just rooted their phones.  What apps, ROMs, benefits, etc. do I get from rooting?  What should I be doing now?</p>\n',
 'Id': '1',
 'ParentId': '',
 'Score': '286',
 'Tags': '<rooting><root-access>',
 'Title': "I've rooted my phone.  Now what?  What do I gain from rooting?"}

In [6]:
for key,value in qa2.items():
    if value['AcceptedAnswerId'] is not '':
        itsAnswer = value['AcceptedAnswerId']
        value['AnswerData'] = qa2[itsAnswer]['Body']

In [7]:
len(qa2)

73389

In [8]:
import copy
quesWithAns = copy.deepcopy(qa2)

In [9]:
for key in list(quesWithAns.keys()):
    if quesWithAns[key]['AcceptedAnswerId'] is '':
        del quesWithAns[key]

In [10]:
len(qa2)

73389

In [11]:
len(quesWithAns)

14029

In [12]:
import re
for key,value in quesWithAns.items():
    value['Body'] = re.sub(r"[^a-zA-Z0-9]+", ' ', value['Body']).lower()
    value['AnswerData'] = re.sub(r"[^a-zA-Z0-9]+", ' ', value['AnswerData']).lower()
    value['Title'] = re.sub(r"[^a-zA-Z0-9]+", ' ', value['Title']).lower()

In [13]:
len(quesWithAns)

14029

In [14]:
quesWithAns['1'] # remove special chars

{'AcceptedAnswerId': '13',
 'AnswerData': ' h1 strong things that require root strong h1 h2 root file explorers h2 p you can remount your code system code directory read write which will allow you to a href http wiki cyanogenmod org w barebones rel nofollow noreferrer remove carrier shipped applications a you don t like and so on root file explorers can allow you to view all folders on your phone such as code data code which is normally protected p ul li a href https play google com store apps details id com blogspot superthomaslab mountsystemrorw rel nofollow noreferrer mount system ro rw a li li a href https play google com store apps details id com speedsoftware rootexplorer rel nofollow noreferrer root explorer a li li a href https play google com store apps details id com estrongs android pop rel nofollow noreferrer es file explorer a li ul h2 patching apps h2 p with root access you can patch other apps to gain special access to functions or removal of self protection for some app

In [15]:
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz ' # space is included in whitelist
EN_BLACKLIST = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\''

#FILENAME = 'data/chat.txt'

limit = {
        'maxq' : 20,
        'minq' : 0,
        'maxa' : 80,
        'mina' : 3
        }

UNK = 'unk'
VOCAB_SIZE = 6000

import random
import sys

import nltk
import itertools
from collections import defaultdict

import numpy as np

import pickle


In [16]:
quesWithAns['1']['AnswerData']

' h1 strong things that require root strong h1 h2 root file explorers h2 p you can remount your code system code directory read write which will allow you to a href http wiki cyanogenmod org w barebones rel nofollow noreferrer remove carrier shipped applications a you don t like and so on root file explorers can allow you to view all folders on your phone such as code data code which is normally protected p ul li a href https play google com store apps details id com blogspot superthomaslab mountsystemrorw rel nofollow noreferrer mount system ro rw a li li a href https play google com store apps details id com speedsoftware rootexplorer rel nofollow noreferrer root explorer a li li a href https play google com store apps details id com estrongs android pop rel nofollow noreferrer es file explorer a li ul h2 patching apps h2 p with root access you can patch other apps to gain special access to functions or removal of self protection for some apps p ul li a href https in appstore com and

In [17]:
quesWithAns['1']['Body']

' p this is a common question by those who have just rooted their phones what apps roms benefits etc do i get from rooting what should i be doing now p '

In [18]:
quesWithAns['1']['Title']

'i ve rooted my phone now what what do i gain from rooting '

In [19]:
def filter_data(sequences):
    filtered_q, filtered_a = [], []
    raw_data_len = len(sequences)//2
    for key, value in sequences.items():
        qlen, alen = sequences[key]['Title'].split(' '), sequences[key]['AnswerData'].split(' ')
        qlen = [e for e in qlen if len(e)>1 and e not in ('h1','h2','h3')]
        alen = [e for e in alen if len(e)>1 and e not in ('h1','h2','h3')]
        if len(qlen) <= limit['maxq'] and len(alen) <= limit['maxa']: # limit length of que and ans
            filtered_q.append(qlen)
            filtered_a.append(alen)
        #break
    return filtered_q, filtered_a

In [20]:
print(type(quesWithAns))
qlines, alines = filter_data(quesWithAns)
print(qlines)
print(alines)

<class 'dict'>


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [21]:
# convert list of [lines of text] into list of [list of words ]
#print('\n>> Segment lines into words')
#qtokenized = [ wordlist.split(' ') for wordlist in qlines ]
#atokenized = [ wordlist.split(' ') for wordlist in alines ]

In [22]:
def index_(tokenized_sentences, vocab_size):
    # get frequency distribution
    freq_dist = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    # get vocabulary of 'vocab_size' most used words
    vocab = freq_dist.most_common(vocab_size)
    # index2word
    index2word = ['_'] + [UNK] + [ x[0] for x in vocab ]
    # word2index
    word2index = dict([(w,i) for i,w in enumerate(index2word)] )
    return index2word, word2index, freq_dist

In [23]:
 # indexing -> idx2w, w2idx : en/ta
print('\n >> Index words')
idx2w, w2idx, freq_dist = index_( qlines + alines, vocab_size=VOCAB_SIZE)
print(idx2w)
print(w2idx)
print(freq_dist)


 >> Index words
['_', 'unk', 'the', 'to', 'you', 'it', 'and', 'is', 'on', 'in', 'com', 'android', 'for', 'code', 'can', 'of', 'that', 'this', 'google', 'href', 'your', 'app', 'rel', 'nofollow', 'with', 'phone', 'from', 'not', 'how', 'apps', 'my', 'if', 'be', 'https', 'li', 'have', 'as', 'http', 'device', 'or', 'an', 'there', 'but', 'will', 'strong', 'are', 'settings', 'do', 'play', 'use', 'data', 'by', 'then', 'was', 'when', 'all', 'store', 'so', 'id', 'screen', 'which', 'details', 'no', 'just', 'only', 'using', 'up', 'one', 'after', 'what', 'has', 'should', 'em', 'www', 'at', 'get', 'galaxy', 'problem', 'like', 'file', 'go', 'does', 'system', 'samsung', 'any', 'try', 'root', 'install', 'amp', 'update', 'need', 'nexus', 'also', 'out', 'way', 'market', 'work', 'version', 'some', 'now', 'usb', 'see', 'card', 'don', 'other', 'here', 'they', 'rom', 'pre', 'account', 'would', 'download', 'off', 'noreferrer', 'sd', 'gt', 'files', 'button', 'same', 'option', 'issue', 'set', 'time', 'mode', '

In [24]:
def zero_pad(qtokenized, atokenized, w2idx):
    # num of rows
    data_len = len(qtokenized)

    # numpy arrays to store indices
    idx_q = np.zeros([data_len, limit['maxq']], dtype=np.int32) 
    idx_a = np.zeros([data_len, limit['maxa']], dtype=np.int32)

    for i in range(data_len):
        q_indices = pad_seq(qtokenized[i], w2idx, limit['maxq'])
        a_indices = pad_seq(atokenized[i], w2idx, limit['maxa'])

        #print(len(idx_q[i]), len(q_indices))
        #print(len(idx_a[i]), len(a_indices))
        idx_q[i] = np.array(q_indices)
        idx_a[i] = np.array(a_indices)

    return idx_q, idx_a

In [25]:
def pad_seq(seq, lookup, maxlen):
    indices = []
    for word in seq:
        if word in lookup:
            indices.append(lookup[word])
        else:
            indices.append(lookup[UNK])
    return indices + [0]*(maxlen - len(seq))

In [26]:
print('\n >> Zero Padding')
idx_q, idx_a = zero_pad(qlines, alines, w2idx)
print(idx_q)


 >> Zero Padding
[[ 146  241  212 ...,    0    0    0]
 [  28   47  423 ...,    0    0    0]
 [  28   47  319 ...,    0    0    0]
 ..., 
 [  28    3  138 ...,    0    0    0]
 [5928 2428    9 ...,    0    0    0]
 [  28    3 1039 ...,    0    0    0]]


In [27]:
print('\n >> Save numpy arrays to disk')
# save them
np.save('idx_q.npy', idx_q)
np.save('idx_a.npy', idx_a)

# let us now save the necessary dictionaries
metadata = {
        'w2idx' : w2idx,
        'idx2w' : idx2w,
        'limit' : limit,
        'freq_dist' : freq_dist
            }

# write to disk : data control dictionaries
with open('metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)


 >> Save numpy arrays to disk


In [28]:
import numpy as np
from random import sample

'''
 split data into train (70%), test (15%) and valid(15%)
    return tuple( (trainX, trainY), (testX,testY), (validX,validY) )

'''
def split_dataset(x, y, ratio = [0.7, 0.15, 0.15] ):
    # number of examples
    data_len = len(x)
    lens = [ int(data_len*item) for item in ratio ]

    trainX, trainY = x[:lens[0]], y[:lens[0]]
    testX, testY = x[lens[0]:lens[0]+lens[1]], y[lens[0]:lens[0]+lens[1]]
    validX, validY = x[-lens[-1]:], y[-lens[-1]:]

    return (trainX,trainY), (testX,testY), (validX,validY)


'''
 generate batches from dataset
    yield (x_gen, y_gen)

    TODO : fix needed

'''
def batch_gen(x, y, batch_size):
    # infinite while
    while True:
        for i in range(0, len(x), batch_size):
            if (i+1)*batch_size < len(x):
                yield x[i : (i+1)*batch_size ].T, y[i : (i+1)*batch_size ].T

'''
 generate batches, by random sampling a bunch of items
    yield (x_gen, y_gen)

'''
def rand_batch_gen(x, y, batch_size):
    while True:
        sample_idx = sample(list(np.arange(len(x))), batch_size)
        yield x[sample_idx].T, y[sample_idx].T

#'''
# convert indices of alphabets into a string (word)
#    return str(word)
#
#'''
#def decode_word(alpha_seq, idx2alpha):
#    return ''.join([ idx2alpha[alpha] for alpha in alpha_seq if alpha ])
#
#
#'''
# convert indices of phonemes into list of phonemes (as string)
#    return str(phoneme_list)
#
#'''
#def decode_phonemes(pho_seq, idx2pho):
#    return ' '.join( [ idx2pho[pho] for pho in pho_seq if pho ])


'''
 a generic decode function 
    inputs : sequence, lookup

'''
def decode(sequence, lookup, separator=''): # 0 used for padding, is ignored
    return separator.join([ lookup[element] for element in sequence if element ])


In [43]:
import tensorflow as tf
import numpy as np
import sys

class Seq2Seq(object):
    def __init__(self, xseq_len, yseq_len, 
            xvocab_size, yvocab_size,
            emb_dim, num_layers, ckpt_path,
            lr=0.0001, 
            epochs=100000, model_name='seq2seq_model'):

        # attach these arguments to self
        self.xseq_len = xseq_len
        self.yseq_len = yseq_len
        self.ckpt_path = ckpt_path
        self.epochs = epochs
        self.model_name = model_name


        # build thy graph
        #  attach any part of the graph that needs to be exposed, to the self
        def __graph__():

            # placeholders
            tf.reset_default_graph()
            #  encoder inputs : list of indices of length xseq_len
            self.enc_ip = [ tf.placeholder(shape=[None,], 
                            dtype=tf.int64, 
                            name='ei_{}'.format(t)) for t in range(xseq_len) ]

            #  labels that represent the real outputs
            self.labels = [ tf.placeholder(shape=[None,], 
                            dtype=tf.int64, 
                            name='ei_{}'.format(t)) for t in range(yseq_len) ]

            #  decoder inputs : 'GO' + [ y1, y2, ... y_t-1 ]
            self.dec_ip = [ tf.zeros_like(self.enc_ip[0], dtype=tf.int64, name='GO') ] + self.labels[:-1]


            # Basic LSTM cell wrapped in Dropout Wrapper
            self.keep_prob = tf.placeholder(tf.float32)
            # define the basic cell
            basic_cell = tf.contrib.rnn.DropoutWrapper(
                    tf.contrib.rnn.BasicLSTMCell(emb_dim, state_is_tuple=True),
                    output_keep_prob=self.keep_prob)
            # stack cells together : n layered model
            stacked_lstm = tf.contrib.rnn.MultiRNNCell([basic_cell]*num_layers, state_is_tuple=True)


            # for parameter sharing between training model
            #  and testing model
            with tf.variable_scope('decoder') as scope:
                # build the seq2seq model 
                #  inputs : encoder, decoder inputs, LSTM cell type, vocabulary sizes, embedding dimensions
                self.decode_outputs, self.decode_states = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(self.enc_ip,self.dec_ip, stacked_lstm,
                                                    xvocab_size, yvocab_size, emb_dim)
                # share parameters
                scope.reuse_variables()
                # testing model, where output of previous timestep is fed as input 
                #  to the next timestep
                self.decode_outputs_test, self.decode_states_test = tf.contrib.legacy_seq2seq.embedding_rnn_seq2seq(
                    self.enc_ip, self.dec_ip, stacked_lstm, xvocab_size, yvocab_size,emb_dim,feed_previous=True)

            # now, for training,
            #  build loss function

            # weighted loss
            #  TODO : add parameter hint
            loss_weights = [ tf.ones_like(label, dtype=tf.float32) for label in self.labels ]
            self.loss = tf.contrib.legacy_seq2seq.sequence_loss(self.decode_outputs, self.labels, loss_weights, yvocab_size)
            # train op to minimize the loss
            self.train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(self.loss)

        sys.stdout.write('<log> Building Graph ')
        # build comput graph
        __graph__()
        sys.stdout.write('</log>')



    '''
        Training and Evaluation
    '''

    # get the feed dictionary
    def get_feed(self, X, Y, keep_prob):
        feed_dict = {self.enc_ip[t]: X[t] for t in range(self.xseq_len)}
        feed_dict.update({self.labels[t]: Y[t] for t in range(self.yseq_len)})
        feed_dict[self.keep_prob] = keep_prob # dropout prob
        return feed_dict

    # run one batch for training
    def train_batch(self, sess, train_batch_gen):
        # get batches
        batchX, batchY = train_batch_gen.__next__()
        # build feed
        feed_dict = self.get_feed(batchX, batchY, keep_prob=0.5)
        _, loss_v = sess.run([self.train_op, self.loss], feed_dict)
        return loss_v

    def eval_step(self, sess, eval_batch_gen):
        # get batches
        batchX, batchY = eval_batch_gen.__next__()
        # build feed
        feed_dict = self.get_feed(batchX, batchY, keep_prob=1.)
        loss_v, dec_op_v = sess.run([self.loss, self.decode_outputs_test], feed_dict)
        # dec_op_v is a list; also need to transpose 0,1 indices 
        #  (interchange batch_size and timesteps dimensions
        dec_op_v = np.array(dec_op_v).transpose([1,0,2])
        return loss_v, dec_op_v, batchX, batchY

    # evaluate 'num_batches' batches
    def eval_batches(self, sess, eval_batch_gen, num_batches):
        losses = []
        for i in range(num_batches):
            loss_v, dec_op_v, batchX, batchY = self.eval_step(sess, eval_batch_gen)
            losses.append(loss_v)
        return np.mean(losses)

    # finally the train function that
    #  runs the train_op in a session
    #   evaluates on valid set periodically
    #    prints statistics
    def train(self, train_set, valid_set, sess=None ):
        
        # we need to save the model periodically
        saver = tf.train.Saver()

        # if no session is given
        if not sess:
            # create a session
            sess = tf.Session()
            # init all variables
            sess.run(tf.global_variables_initializer())

        sys.stdout.write('\n<log> Training started </log>\n')
        # run M epochs
        for i in range(self.epochs):
            try:
                self.train_batch(sess, train_set)
                if i and i% (self.epochs//100) == 0: # TODO : make this tunable by the user
                    # save model to disk
                    saver.save(sess, self.ckpt_path + self.model_name + '.ckpt', global_step=i)
                    # evaluate to get validation loss
                    val_loss = self.eval_batches(sess, valid_set, 16) # TODO : and this
                    # print stats
                    print('\nModel saved to disk at iteration #{}'.format(i))
                    print('val   loss : {0:.6f}'.format(val_loss))
                    sys.stdout.flush()
            except KeyboardInterrupt: # this will most definitely happen, so handle it
                print('Interrupted by user at iteration {}'.format(i))
                self.session = sess
                return sess

    def restore_last_session(self):
        saver = tf.train.Saver()
        # create a session
        sess = tf.Session()
        # get checkpoint state
        ckpt = tf.train.get_checkpoint_state(self.ckpt_path)
        # restore session
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        # return to user
        return sess

    # prediction
    def predict(self, sess, X):
        feed_dict = {self.enc_ip[t]: X[t] for t in range(self.xseq_len)}
        feed_dict[self.keep_prob] = 1.
        dec_op_v = sess.run(self.decode_outputs_test, feed_dict)
        # dec_op_v is a list; also need to transpose 0,1 indices 
        #  (interchange batch_size and timesteps dimensions
        dec_op_v = np.array(dec_op_v).transpose([1,0,2])
        # return the index of item with highest probability
        return np.argmax(dec_op_v, axis=2)


In [44]:
# In[1]:
import tensorflow as tf
import numpy as np

# preprocessed data
#from datasets.twitter import data
#import data_utils

# load data from pickle and npy files
#metadata, idx_q, idx_a = data.load_data(PATH='')
(trainX, trainY), (testX, testY), (validX, validY) = split_dataset(idx_q, idx_a)

# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 32
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

#import seq2seq_wrapper

# In[7]:

#model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
model = Seq2Seq(xseq_len=xseq_len,
               yseq_len=yseq_len,
               xvocab_size=xvocab_size,
               yvocab_size=yvocab_size,
               ckpt_path='/home/neil/Neil_Work/MS_SJSU/295A/chatbot-android/project/ml/ckpt_path',
               emb_dim=emb_dim,
               num_layers=3
               )


# In[8]:

val_batch_gen = data_utils.rand_batch_gen(validX, validY, 32)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)


# In[9]:
sess = model.restore_last_session()
sess = model.train(train_batch_gen, val_batch_gen)

<log> Building Graph 

TypeError: can't pickle _thread.RLock objects